In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
# from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pickle

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Set Parameters
date_time_str = "2023/09/05/"
save_csv = False
name_cs_str = "CS1"      
bo_meth_list = ["A1", "B1", "A2", "C2"]
ep_meth_list = ["CONSTANT", "DECAY", "BOYLE", "JASRASARIA"]
study_id = "EP"

#Run EP Bias Experiment Analysis
analyze_ep_sep_fact_study(date_time_str, bo_meth_list, ep_meth_list, study_id, name_cs_str, save_csv)

  BO Method Best EP Method       Min SSE   Min SSE Act   theta_1   theta_2  \
0        A1          BOYLE  4.276449e-02  1.861867e-01  1.135790 -0.992728   
1        B1          DECAY  9.439487e-02  4.105523e-02  1.061143 -1.010391   
2        A2       CONSTANT  2.360463e-07  1.503054e-06  1.000157 -0.999808   
3        C2          BOYLE  8.030039e-08  9.989756e-07  0.999800 -0.999867   

   BO Restart  BO Iter  
0         2.0      3.0  
1        14.0     31.0  
2         1.0      5.0  
3        10.0      9.0  
[ 1. -1.]


In [ ]:
#Separation factor experiment analysis
date_time_str = "2023/09/06/"
save_csv = False
f = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
precision = 1
sep_fact_list = [f"{number:.{precision}f}" for number in f]
bo_meth_list = ["A1", "B1", "A2"]
study_id = "SF"

#Run Sep Fact Experiment Analysis
analyze_ep_sep_fact_study(date_time_str, bo_meth_list, sep_fact_list, study_id, name_cs_str, save_csv)